# Lab: Vertex AI–Assisted BigQuery Analytics — Example Prompts
**Goal:** Practice moving from simple SQL to complex analytics in BigQuery using *only* carefully engineered prompts with Vertex AI (Gemini).  
**Important:** This notebook contains **prompts only** (no starter code). Paste the prompts into **Vertex AI Studio**, **Vertex AI in Colab Enterprise**, or your chosen chat interface, and then run the generated SQL directly in **BigQuery**. If you decide to automate later, you can ask Vertex AI to convert the winning SQL into a Colab pipeline.

## How to use this prompts-only notebook
1. Open **Vertex AI Studio** (or Gemini in Colab Enterprise chat panel).  
2. Copy a prompt from this notebook and paste it into the model. Do **not** paste any code from here; let the model generate it.  
3. Run the generated SQL in **BigQuery** (Console → BigQuery Studio).  
4. Iterate: refine the prompt when results aren’t what you expect.  
5. Document: capture your final SQL, plus a one-sentence takeaway, in your notes/README.

## Dataset assumptions
Use one of these sources (adjust table paths accordingly):
- **Global Superstore (Kaggle)** loaded into BigQuery (e.g., `[YOUR_PROJECT].superstore_data.sales`)  
- **TheLook eCommerce** public dataset: `bigquery-public-data.thelook_ecommerce`  
If you are using *Global Superstore*, make sure column names match your schema (e.g., `Order_Date`, `Region`, `Category`, `Sub_Category`, `Sales`, `Profit`, `Discount`, `State`, `Customer_ID`, `Ship_Mode`).

---
## Prompting guardrails (quick checklist)
- **Be explicit**: table path, column names, filters, output columns, sort order, and limits.  
- **Ask for runnable SQL**: “Return a BigQuery SQL block only.”  
- **Control cost**: ask for `LIMIT` during exploration and remove it for the final run.  
- **Validate**: request a brief explanation of why each clause is present and how you can sanity-check results.
---

## Install Dependencies

In [ ]:
# Install the Google Cloud BigQuery client library
!pip install google-cloud-bigquery==3.17.0 pandas==2.1.4

# Authenticate your Colab environment
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


## Copy Schema to a dataframe

In [ ]:
from google.cloud import bigquery
import pandas as pd

# Replace with your Google Cloud Project ID
project_id = 'our-rock-471819-h7' # This is derived from your provided table name
dataset_id = 'lab1_foundation'
table_id = 'superstore'

# Construct a BigQuery client object.
client = bigquery.Client(project=project_id)

# Get the table object
table_ref = client.dataset(dataset_id).table(table_id)
table = client.get_table(table_ref)

# Extract schema information
schema_list = []
for field in table.schema:
    schema_list.append({
        'name': field.name,
        'field_type': field.field_type,
        'mode': field.mode,
        'description': field.description
    })

# Convert to Pandas DataFrame
schema_df = pd.DataFrame(schema_list)

# Display the schema DataFrame (optional, for verification)
print("Schema DataFrame created:")
# To see the output, run the code.


Schema DataFrame created:


## CLean Column Names

In [ ]:
# --- 1. Clean the Column Names ---
# Create a 'clean_name' column with standard naming conventions:
# lowercase, with spaces and hyphens replaced by underscores.
schema_df['clean_name'] = schema_df['name'].str.lower().str.replace(' ', '_').str.replace('-', '_')


# --- 2. Generate the Aliases for the SELECT Clause ---
column_expressions = []
for index, row in schema_df.iterrows():
    original_name = row['name']
    clean_name = row['clean_name']

    # If the original name contains a space or special character, it needs to be
    # enclosed in backticks (`) in the SQL statement.
    if ' ' in original_name or '-' in original_name:
        expression = f'`{original_name}` AS {clean_name}'
    else:
        # If the name is already clean, we still alias it for consistency.
        expression = f'{original_name} AS {clean_name}'
    column_expressions.append(expression)

# Join all the individual column expressions into a single, formatted string.
select_clause = ",\n  ".join(column_expressions)


# --- 3. Construct the Final CREATE VIEW Statement ---
new_view_id = 'superstore_clean' # You can change this if you like

create_view_sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{new_view_id}` AS
SELECT
  {select_clause}
FROM
  `{project_id}.{dataset_id}.{table_id}`;
"""

# --- 4. Print the Final SQL ---
print("--- Copy the SQL below and run it in your BigQuery Console ---")
print(create_view_sql)

--- Copy the SQL below and run it in your BigQuery Console ---

CREATE OR REPLACE VIEW `our-rock-471819-h7.lab1_foundation.superstore_clean` AS
SELECT
  `Row ID` AS row_id,
  `Order ID` AS order_id,
  `Order Date` AS order_date,
  `Ship Date` AS ship_date,
  `Ship Mode` AS ship_mode,
  `Customer ID` AS customer_id,
  `Customer Name` AS customer_name,
  Segment AS segment,
  Country_Region AS country_region,
  City AS city,
  State_Province AS state_province,
  `Postal Code` AS postal_code,
  Region AS region,
  `Product ID` AS product_id,
  Category AS category,
  `Sub-Category` AS sub_category,
  `Product Name` AS product_name,
  Sales AS sales,
  Quantity AS quantity,
  Discount AS discount,
  Profit AS profit
FROM
  `our-rock-471819-h7.lab1_foundation.superstore`;



## Generate View with standard column naming convention

In [ ]:
# Execute the CREATE VIEW SQL query
try:
    query_job = client.query(create_view_sql)  # API request
    query_job.result()  # Waits for the query to finish
    print(f"View '{new_view_id}' created/replaced successfully in dataset '{dataset_id}'.")
except Exception as e:
    print(f"An error occurred while creating the view: {e}")

# Now, let's print 10 rows from the newly created view to verify
print(f"\n--- First 10 rows from the new view '{new_view_id}' ---")
try:
    # Construct a reference to the new view
    view_table_ref = client.dataset(dataset_id).table(new_view_id)

    # Fetch the first 10 rows
    rows = client.list_rows(view_table_ref, max_results=10)

    # Print header
    print(" | ".join([field.name for field in rows.schema]))
    print("-" * 80) # Separator

    # Print rows
    for row in rows:
        print(" | ".join([str(item) for item in row.values()]))

except Exception as e:
    print(f"An error occurred while fetching rows from the view: {e}")



View 'superstore_clean' created/replaced successfully in dataset 'lab1_foundation'.

--- First 10 rows from the new view 'superstore_clean' ---
row_id | order_id | order_date | ship_date | ship_mode | customer_id | customer_name | segment | country_region | city | state_province | postal_code | region | product_id | category | sub_category | product_name | sales | quantity | discount | profit
--------------------------------------------------------------------------------
An error occurred while fetching rows from the view: 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/our-rock-471819-h7/datasets/lab1_foundation/tables/superstore_clean/data?maxResults=10&formatOptions.useInt64Timestamp=True&prettyPrint=false: Cannot list a table of type VIEW.


In [ ]:
# This assumes your 'client' object from the previous cell is still active
# and correctly authenticated.

print("✅ Step 1: Defining the query string...")

query_string = """
SELECT
  order_id,
  customer_name,
  product_name,
  sales,
  profit
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean`
LIMIT 10;
"""

print("✅ Step 2: Sending the query to BigQuery. This may take a moment...")

# Use a try-except block to catch potential errors
try:
    query_job = client.query(query_string)

    print("✅ Step 3: Waiting for query to complete and fetching results...")
    results_df = query_job.to_dataframe()

    print(f"✅ Step 4: Query finished. Found {len(results_df)} rows.")

    if results_df.empty:
        print("\n⚠️ The query ran successfully but returned an empty result. Please double-check that your 'superstore_clean' view exists and the original table has data.")
    else:
        print("\n--- Displaying Results ---")
        display(results_df)

except Exception as e:
    print(f"\n❌ An error occurred: {e}")

✅ Step 1: Defining the query string...
✅ Step 2: Sending the query to BigQuery. This may take a moment...
✅ Step 3: Waiting for query to complete and fetching results...
✅ Step 4: Query finished. Found 10 rows.

--- Displaying Results ---


,order_id,customer_name,product_name,sales,profit
0,US-2022-154900,Sung Shariari,Avery 518,3.15,1.5120
1,US-2022-154900,Sung Shariari,Adams Telephone Message Book W/Dividers/Space ...,22.72,10.2240
2,US-2023-152415,Patrick O'Donnell,"C-Line Magnetic Cubicle Keepers, Clear Polypro...",14.82,6.2244
3,US-2023-152415,Patrick O'Donnell,"Howard Miller 14-1/2"" Diameter Chrome Round Wa...",191.82,61.3824
4,US-2023-153269,Pamela Stobb,"Situations Contoured Folding Chairs, 4/Set",354.90,88.7250
5,US-2023-153269,Pamela Stobb,GBC Binding covers,51.80,23.3100
6,US-2023-153269,Pamela Stobb,Xerox 193,17.94,8.7906
7,US-2023-153269,Pamela Stobb,"Personal Folder Holder, Ebony",11.21,3.3630
8,US-2021-166086,Carol Triggs,"Lifetime Advantage Folding Chairs, 4/Carton",872.32,244.2496
9,US-2021-166086,Carol Triggs,Balt Split Level Computer Training Table,194.25,-38.8500


## Part A — SQL Warm‑Up (SELECT, WHERE, ORDER BY, LIMIT, DISTINCT)
**Aim:** Build confidence with precise, unambiguous prompts that yield clean, runnable SQL.

### A1. Unique values (DISTINCT)
**Prompt (paste in Vertex AI):**
```
Act as a senior BigQuery analyst. Produce a **single runnable BigQuery SQL** (no commentary) for:
- Task: List all unique `Sub_Category` values sold in the 'West' region.
- Table: `our-rock-471819-h7.lab1_foundation.superstore_clean`
- Filter: `Region = 'West'`
- Output: a single column named `Sub_Category`
- Sort: alphabetically A→Z
- Add: `LIMIT 100` to control cost during exploration.
```
**Reflection:** Did the result match your expectations? If not, what ambiguity in your prompt might have caused the mismatch?

Yes the results were the same.

In [20]:
query_string = """
SELECT
    DISTINCT sub_category
FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
WHERE
    region = 'West'
ORDER BY
    sub_category ASC
LIMIT 100
"""
try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,sub_category
0,Accessories
1,Appliances
2,Art
3,Binders
4,Bookcases
5,Chairs
6,Copiers
7,Envelopes
8,Fasteners
9,Furnishings


### A2. Top‑N by metric (ORDER BY … DESC)
**Prompt:**
```
BigQuery SQL only.
Task: Return the top 10 customers by total profit.
Table: `our-rock-471819-h7.lab1_foundation.superstore_clean`
Columns used: `Customer_ID`, `Profit`
Output columns: `Customer_ID`, `total_profit`
Logic: SUM Profit per customer, order by `total_profit` DESC
Add `LIMIT 10`.
```
**Tip:** If your schema uses different identifiers (e.g., `Customer Name`), restate column names explicitly.

In [21]:
query_string = """
SELECT
    customer_id AS Customer_ID,
    SUM(profit) AS total_profit
FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
GROUP BY
    customer_id
ORDER BY
    total_profit DESC
LIMIT 10
"""
try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,Customer_ID,total_profit
0,TC-20980,8981.3239
1,RB-19360,6976.0959
2,SC-20095,5757.4119
3,HL-15040,5622.4292
4,AB-10105,5444.8055
5,TA-21385,4703.7883
6,CM-12385,3899.8904
7,KD-16495,3038.6254
8,AR-10540,2884.6208
9,DR-12940,2869.0760


### A3. Basic filtering (WHERE) + sanity checks
**Prompt:**
```
BigQuery SQL only.
Task: Count orders shipped with each `Ship_Mode`, but only for orders in the 'Technology' category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Ship_Mode`, `order_count`
Logic: COUNT(*) grouped by `Ship_Mode`
Sort by `order_count` DESC
```
**Validation ask:** “Also list two quick sanity checks to verify the numbers.”

1. Total Count Check: Run a separate query to COUNT(*) all rows where category = 'Technology'. The sum of order_count from the above query should equal this total count.
2. Ship Mode Verification: Pick one Ship_Mode from the results (e.g., 'Standard Class') and run a query to COUNT(*) rows where category = 'Technology' AND ship_mode = 'Standard Class'. This count should match the order_count for 'Standard Class' in the original result.

In [22]:
query_string = """
SELECT
    ship_mode AS Ship_Mode,
    COUNT(*) AS order_count
FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
WHERE
    category = 'Technology'
GROUP BY
    ship_mode
ORDER BY
    order_count DESC
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,Ship_Mode,order_count
0,Standard Class,1096
1,Second Class,366
2,First Class,301
3,Same Day,102


## Part B — Grouped Analytics (GROUP BY, HAVING)
**Aim:** Turn raw facts into grouped metrics and filtered aggregations.

### B1. KPI aggregation with WHERE + GROUP BY
**Prompt:**
```
BigQuery SQL only.
Task: Compute monthly revenue for the last 12 full months.
Table: `[YOUR_PROJECT].superstore_data.sales`
Assume: `Order_Date` is a DATE or TIMESTAMP column named exactly `Order_Date`.
Output: `year_month` (YYYY-MM format), `monthly_revenue`
Logic: Truncate date to month, SUM `Sales`, filter to last 12 full months.
Sort by `year_month` ascending.
Include a `LIMIT` safeguard for exploration.
```

In [23]:
query_string = """
SELECT
    FORMAT_DATE('%Y-%m', order_date) AS year_month,
    SUM(sales) AS monthly_revenue
FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
WHERE
    order_date >= DATE_TRUNC(DATE_SUB((SELECT MAX(order_date) FROM `our-rock-471819-h7.lab1_foundation.superstore_clean`), INTERVAL 11 MONTH), MONTH)
GROUP BY
    year_month
ORDER BY
    year_month ASC
LIMIT 12;
"""
try:
    # This assumes 'client' is an authenticated BigQuery client from a previous cell.
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()

    print(f"Query executed successfully. Found {len(results_df)} months of data.")
    display(results_df)

except Exception as e:
    print(f"An error occurred while executing the query: {e}")

Query executed successfully. Found 12 months of data.


,year_month,monthly_revenue
0,2024-01,44259.2140
1,2024-02,20301.1334
2,2024-03,60728.4808
3,2024-04,36779.0361
4,2024-05,45155.4822
5,2024-06,53056.0777
6,2024-07,45989.4960
7,2024-08,64129.7580
8,2024-09,88064.5320
9,2024-10,83474.7832


### B2. Post‑aggregation filter (HAVING)
**Prompt:**
```
BigQuery SQL only.
Task: Find sub-categories whose total profit over the entire dataset is negative.
Table: `[YOUR_PROJECT].superstore_data.sales`
Output: `Sub_Category`, `total_profit`
Logic: SUM `Profit` GROUP BY `Sub_Category`, HAVING SUM(Profit) < 0
Sort by `total_profit` ASC (most negative first).
```
**Why HAVING?** Ask the model to include a 1-sentence explanation of why HAVING is used instead of WHERE here.

In [24]:
query_string = """
SELECT
    sub_category,
    SUM(profit) AS total_profit
FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
GROUP BY
    sub_category
HAVING
    SUM(profit) < 0
ORDER BY
    total_profit ASC
"""
try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,sub_category,total_profit
0,Tables,-17753.2061
1,Bookcases,-3632.0736
2,Supplies,-1171.3945


**Why HAVING is used instead of WHERE:**
`HAVING` is necessary here because it filters the results *after* the `SUM(Profit)` aggregation has been calculated for each group; in contrast, `WHERE` filters individual rows *before* they are grouped and aggregated.

## Part C — Joins (dimension enrichment)
**Aim:** Use joins to enhance facts with attributes.

### C1. Join facts to a small dimension
*(If you have a customer or product dimension in your schema, use it. Otherwise, request a synthetic example.)*  
**Prompt:**
```
BigQuery SQL only.
Task: Join the sales table to a product dimension to report `Product_ID`, `Product_Name`, and total sales.
Tables: `[YOUR_PROJECT].superstore_data.sales` as s, `[YOUR_PROJECT].superstore_data.products` as p
Join key: `s.Product_ID = p.Product_ID`
Output: `Product_ID`, `Product_Name`, `total_sales`
Sort by `total_sales` DESC
```
**If you lack a dimension table:** Ask the model how to simulate one temporarily via a CTE.

In [25]:
# The prompt asks to demonstrate a JOIN.
# Since product_name is already in our main table, we'll simulate a
# product dimension table using a CTE as suggested in the lab instructions.

query_string = """
WITH product_dimension AS (
  -- Create a distinct list of products to act as our dimension table
  SELECT DISTINCT
    product_id,
    product_name
  FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
)
SELECT
  s.product_id,
  p.product_name,
  SUM(s.sales) AS total_sales
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean` AS s
JOIN
  product_dimension AS p ON s.product_id = p.product_id
GROUP BY
  s.product_id,
  p.product_name
ORDER BY
  total_sales DESC
LIMIT 20; -- Limiting results for demonstration
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()

    print("--- Top 20 Products by Total Sales ---")
    display(results_df)

except Exception as e:
    print(f"An error occurred: {e}")

--- Top 20 Products by Total Sales ---


,product_id,product_name,total_sales
0,TEC-CO-10004722,Canon imageCLASS 2200 Advanced Copier,61599.8240
1,OFF-BI-10003527,Fellowes PB500 Electric Punch Plastic Comb Bin...,27453.3840
2,TEC-MA-10002412,Cisco TelePresence System EX90 Videoconferenci...,22638.4800
3,FUR-CH-10002024,HON 5400 Series Task Chairs for Big and Tall,21870.5760
4,OFF-BI-10001359,GBC DocuBind TL300 Electric Binding System,19823.4790
5,OFF-BI-10000545,GBC Ibimaster 500 Manual ProClick Binding System,19024.5000
6,TEC-CO-10001449,Hewlett Packard LaserJet 3310 Copier,18839.6860
7,TEC-MA-10001127,HP Designjet T520 Inkjet Large Format Printer ...,18374.8950
8,OFF-BI-10004995,GBC DocuBind P400 Electric Binding System,17965.0680
9,OFF-SU-10000151,High Speed Automatic Electric Letter Opener,17030.3120


## Part D — Common Table Expressions (CTEs)
**Aim:** Make complex logic readable and testable in steps.

### D1. Multi‑step ranking with CTEs
**Prompt:**
```
BigQuery SQL only.
Goal: Within each `Region`, rank states by total sales and return top 3 per region.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE 1 (`state_sales`): SUM(Sales) by `Region`, `State`
CTE 2 (`ranked_state_sales`): Add `RANK() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as `sales_rank`
Final SELECT: rows where `sales_rank <= 3`
Output columns: `Region`, `State`, `total_sales`, `sales_rank`
Sort: by `Region`, then `sales_rank`
```
**Ask for**: a one-paragraph explanation of each step, then **provide only the final runnable SQL**.

Here is an explanation of each step in the query:

*   **CTE 1 (`state_sales`)**: The first step, `state_sales`, creates a temporary table that aggregates the data to calculate the total sales for each state within every region. This is achieved by grouping the data by both `region` and `state_province` and summing the `sales`.

*   **CTE 2 (`ranked_state_sales`)**: The second step, `ranked_state_sales`, takes the summary from the first CTE and applies a `RANK()` window function. The `PARTITION BY region` clause is crucial as it tells the function to restart the ranking for each new region, while `ORDER BY total_sales DESC` assigns a rank of 1 to the state with the highest sales in that partition.

*   **Final SELECT**: The final query selects the desired columns from the `ranked_state_sales` CTE and filters the results to only include rows where the `sales_rank` is less than or equal to 3. This effectively isolates the top three performing states in each region, which are then sorted for a clear and organized final output.

In [26]:
query_string = """
WITH
  state_sales AS (
    -- CTE 1: Sum sales by state within each region
    SELECT
      region,
      state_province AS state,
      SUM(sales) AS total_sales
    FROM
      `our-rock-471819-h7.lab1_foundation.superstore_clean`
    GROUP BY
      region,
      state_province
  ),
  ranked_state_sales AS (
    -- CTE 2: Rank states within each region based on sales
    SELECT
      region,
      state,
      total_sales,
      RANK() OVER (PARTITION BY region ORDER BY total_sales DESC) AS sales_rank
    FROM
      state_sales
  )
-- Final SELECT: Filter for the top 3 states per region
SELECT
  region,
  state,
  total_sales,
  sales_rank
FROM
  ranked_state_sales
WHERE
  sales_rank <= 3
ORDER BY
  region,
  sales_rank;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,region,state,total_sales,sales_rank
0,Central,Texas,170188.0458,1
1,Central,Illinois,80166.1010,2
2,Central,Michigan,76269.6140,3
3,East,New York,310876.2710,1
4,East,Pennsylvania,116511.9140,2
5,East,Ohio,78258.1360,3
6,South,Florida,89473.7080,1
7,South,Virginia,70636.7200,2
8,South,North Carolina,55603.1640,3
9,West,California,457687.6315,1


### D2. Time‑boxed “most improved” analysis
**Prompt:**
```
BigQuery SQL only.
Goal: Identify the top 5 sub-categories with the largest YoY revenue increase from 2023 to 2024.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `yr_sales`: SUM(Sales) by `Sub_Category` and `year` extracted from `Order_Date`
Final: pivot or self-join to compute delta (2024 minus 2023) as `yoy_delta`
Output: `Sub_Category`, `sales_2023`, `sales_2024`, `yoy_delta`
Order by `yoy_delta` DESC
Limit 5
```
**Validation:** Ask the model for two quick failure modes (e.g., missing years) and how to handle them.

### Failure Modes and Handling

Here are two potential issues with YoY analysis and how to address them:

1.  **Missing Year Data**: A sub-category might have sales in 2024 but not in 2023 (a new product line), or vice versa. A simple inner join between two year-specific queries would completely miss these important cases. **Solution**: The query below handles this by using conditional aggregation (`SUM(IF(...))`) within a single pass. This correctly treats a missing year's sales as zero for a given sub-category, ensuring new or discontinued lines are properly included in the delta calculation.

2.  **NULL Values in Calculations**: If a sub-category has no sales in one of the years, the `SUM` might result in `NULL`. When you calculate the delta (`sales_2024 - sales_2023`), performing arithmetic with `NULL` (e.g., `100 - NULL`) results in `NULL`, which would incorrectly exclude that sub-category from the final ranking. **Solution**: Wrap the aggregated sales figures in `IFNULL(..., 0)`. This ensures that any `NULL` sum is treated as a zero, allowing for a correct delta calculation across all sub-categories.

In [27]:
query_string = """
WITH yr_sales AS (
  -- First, calculate total sales for each sub-category for the specific years.
  SELECT
    EXTRACT(YEAR FROM order_date) AS sales_year,
    sub_category,
    SUM(sales) AS total_sales
  FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
  WHERE
    EXTRACT(YEAR FROM order_date) IN (2023, 2024)
  GROUP BY
    sales_year,
    sub_category
)
-- Now, pivot the data to calculate the YoY change.
SELECT
  sub_category,
  IFNULL(sales_2023, 0) AS sales_2023,
  IFNULL(sales_2024, 0) AS sales_2024,
  (IFNULL(sales_2024, 0) - IFNULL(sales_2023, 0)) AS yoy_delta
FROM (
  SELECT
    sub_category,
    SUM(IF(sales_year = 2023, total_sales, 0)) AS sales_2023,
    SUM(IF(sales_year = 2024, total_sales, 0)) AS sales_2024
  FROM
    yr_sales
  GROUP BY
    sub_category
)
ORDER BY
  yoy_delta DESC
LIMIT 5;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print("--- Top 5 Sub-Categories by YoY Revenue Increase (2023-2024) ---")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

--- Top 5 Sub-Categories by YoY Revenue Increase (2023-2024) ---


,sub_category,sales_2023,sales_2024,yoy_delta
0,Phones,79178.010,105668.392,26490.382
1,Binders,51580.133,73651.269,22071.136
2,Accessories,41895.854,59946.232,18050.378
3,Appliances,26164.235,43111.156,16946.921
4,Copiers,49599.410,62916.668,13317.258


## Part E — Window Functions (ROW_NUMBER, RANK, DENSE_RANK, LAG/LEAD, moving averages)
**Aim:** Compare rows across partitions and time; compute trends and ranks without collapsing rows.

### E1. Top product per region (ROW_NUMBER)
**Prompt:**
```
BigQuery SQL only.
Task: For each `Region`, return only the single highest-revenue `Sub_Category`.
Table: `[YOUR_PROJECT].superstore_data.sales`
CTE `subcat_sales`: SUM(Sales) by `Region`, `Sub_Category`
Add `ROW_NUMBER() OVER (PARTITION BY Region ORDER BY total_sales DESC)` as rn
Final: filter `rn = 1`
Output: `Region`, `Sub_Category`, `total_sales`
Sort by `Region`
```
**Why `ROW_NUMBER` instead of `RANK`?** Ask the model to add a 2-sentence contrast.

### Why `ROW_NUMBER` instead of `RANK`?
`ROW_NUMBER()` is used here because the goal is to return exactly one sub-category per region, and it guarantees this by assigning a unique, sequential number to each row, arbitrarily breaking any ties for the top spot. In contrast, `RANK()` assigns the same rank to tied values, which could result in returning multiple 'top' sub-categories if they had identical sales totals.

In [28]:
query_string = """
WITH
  subcat_sales AS (
    -- First, aggregate sales by region and sub-category
    SELECT
      region,
      sub_category,
      SUM(sales) AS total_sales
    FROM
      `our-rock-471819-h7.lab1_foundation.superstore_clean`
    GROUP BY
      region,
      sub_category
  ),
  ranked_sales AS (
    -- Then, rank sub-categories within each region
    SELECT
      region,
      sub_category,
      total_sales,
      ROW_NUMBER() OVER (PARTITION BY region ORDER BY total_sales DESC) AS rn
    FROM
      subcat_sales
  )
-- Finally, select only the top-ranked row for each region
SELECT
  region,
  sub_category,
  total_sales
FROM
  ranked_sales
WHERE
  rn = 1
ORDER BY
  region;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

,region,sub_category,total_sales
0,Central,Chairs,85941.142
1,East,Phones,102361.812
2,South,Phones,58304.438
3,West,Chairs,107134.978


### E2. YoY growth with LAG
**Prompt:**
```
BigQuery SQL only.
Task: Compute year-over-year revenue growth for 'Phones' sub-category.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Filter to `Sub_Category = 'Phones'`
- Aggregate yearly revenue using EXTRACT(YEAR FROM Order_Date)
- Add `LAG(yearly_revenue) OVER (ORDER BY year)` as `prev_revenue`
- Compute `yoy_pct = 100.0 * (yearly_revenue - prev_revenue) / prev_revenue`
Output: `year`, `yearly_revenue`, `prev_revenue`, `yoy_pct`
Sort by `year` ASC
```
**Ask for**: a guard against divide-by-zero or NULL previous year.

In [29]:
query_string = """
WITH
  yearly_phone_sales AS (
    -- Steps 1 & 2: Filter for 'Phones' and aggregate revenue by year
    SELECT
      EXTRACT(YEAR FROM order_date) AS sales_year,
      SUM(sales) AS yearly_revenue
    FROM
      `our-rock-471819-h7.lab1_foundation.superstore_clean`
    WHERE
      sub_category = 'Phones'
    GROUP BY
      sales_year
  ),
  yoy_growth AS (
    -- Step 3: Use LAG to get the previous year's revenue
    SELECT
      sales_year,
      yearly_revenue,
      LAG(yearly_revenue, 1) OVER (ORDER BY sales_year) AS prev_revenue
    FROM
      yearly_phone_sales
  )
-- Step 4: Calculate YoY growth percentage with a divide-by-zero guard
SELECT
  sales_year AS year,
  yearly_revenue,
  prev_revenue,
  -- Use SAFE_DIVIDE to prevent errors when prev_revenue is NULL (for the first year)
  100.0 * SAFE_DIVIDE(yearly_revenue - prev_revenue, prev_revenue) AS yoy_pct
FROM
  yoy_growth
ORDER BY
  year ASC;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print("--- YoY Revenue Growth for 'Phones' ---")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

--- YoY Revenue Growth for 'Phones' ---


,year,yearly_revenue,prev_revenue,yoy_pct
0,2021,78470.826,NaN,NaN
1,2022,68525.412,78470.826,-12.674027
2,2023,79178.010,68525.412,15.545471
3,2024,105668.392,79178.010,33.456741


### E3. 3‑month moving average (MA)
**Prompt:**
```
BigQuery SQL only.
Task: For the 'Corporate' segment, compute a 3-month moving average of monthly revenue.
Table: `[YOUR_PROJECT].superstore_data.sales`
Steps:
- Derive `month` via DATE_TRUNC(Order_Date, MONTH)
- SUM(Sales) per `month`
- Add `AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW)` as `ma_3`
Output: `month`, `monthly_revenue`, `ma_3`
Sort by `month` ASC
```
**Tip:** Ask the model to include a 1‑line cost control note (e.g., restrict date range while iterating).

In [30]:
query_string = """
WITH monthly_corporate_revenue AS (
  -- Step 1 & 2: Filter for 'Corporate' segment and aggregate revenue by month
  SELECT
    DATE_TRUNC(order_date, MONTH) AS month,
    SUM(sales) AS monthly_revenue
  FROM
    `our-rock-471819-h7.lab1_foundation.superstore_clean`
  WHERE
    segment = 'Corporate'
    -- Cost-control note: For exploration, add a date filter like "AND order_date >= 'YYYY-MM-DD'" to limit scanned data.
  GROUP BY
    month
)
-- Step 3: Compute the 3-month moving average
SELECT
  month,
  monthly_revenue,
  AVG(monthly_revenue) OVER (ORDER BY month ROWS BETWEEN 2 PRECEDING AND CURRENT ROW) AS ma_3
FROM
  monthly_corporate_revenue
ORDER BY
  month ASC;
"""

try:
    query_job = client.query(query_string)
    results_df = query_job.to_dataframe()
    print("--- 3-Month Moving Average Revenue for 'Corporate' Segment ---")
    display(results_df)
except Exception as e:
    print(f"An error occurred: {e}")

--- 3-Month Moving Average Revenue for 'Corporate' Segment ---


,month,monthly_revenue,ma_3
0,2021-01-01,1982.6880,1982.688000
1,2021-02-01,1183.6680,1583.178000
2,2021-03-01,11106.7990,4757.718333
3,2021-04-01,14131.7290,8807.398667
4,2021-05-01,9142.0000,11460.176000
5,2021-06-01,3970.9140,9081.547667
6,2021-07-01,10032.9880,7715.300667
7,2021-08-01,7471.2140,7158.372000
8,2021-09-01,16363.1430,11289.115000
9,2021-10-01,13564.9240,12466.427000


## Part F — Debugging & Optimization Prompts
**Aim:** Use the model as a rubber duck for error handling and performance.

### F1. Explain the error, propose a fix
**Prompt:**
```
I ran this BigQuery SQL and got an error:
SELECT
  region,
  sub_category,
  SUM(sales) as total_sales
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean`
GROUP BY
  region
ORDER BY
  region, total_sales DESC;
Error Message:

Error: SELECT list expression references column `sub_category` which is neither grouped nor aggregated at [3:3]
Act as a BigQuery trouble‑shooter.
1) Identify the root cause.
2) Propose the smallest possible fix.
3) Suggest a quick sanity check query to verify the fix.
Return only the corrected SQL and a 2‑sentence rationale.
```

```sql
SELECT
  region,
  sub_category,
  SUM(sales) as total_sales
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean`
GROUP BY
  region,         -- The fix is to add sub_category here
  sub_category
ORDER BY
  region, total_sales DESC;
```

The root cause of the error is that the `sub_category` column was included in the `SELECT` list but was not part of an aggregate function or listed in the `GROUP BY` clause. The fix is to add `sub_category` to the `GROUP BY` clause, which correctly instructs the database to calculate the sum of sales for each unique combination of region and sub-category.

### F2. Reduce cost / improve speed
**Prompt:**
```
Act as a BigQuery cost optimizer.
Given this query (below), list 3 ways to reduce scanned bytes and improve performance without changing the business logic.
[PASTE YOUR SQL HERE]
Prioritize: partition filters, column pruning, pre-aggregations, and temporary results via CTEs.
```

### Query for Optimization:

Let's assume you want to analyze the profitability of all orders for 'Phones' and 'Chairs' over the last four years. A first-pass query might look like this:

```sql
-- Inefficient Query Example
SELECT
  *
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean`
WHERE
  sub_category IN ('Phones', 'Chairs')
  AND EXTRACT(YEAR FROM order_date) BETWEEN 2021 AND 2024;
```

### Three Optimization Recommendations:

Here are three ways to make the above query more performant and cost-effective without changing the business logic:

1.  **Column Pruning (Reduce Data Processed):** The most significant issue is `SELECT *`. This forces BigQuery to read every single column from the table. You should only select the columns you actually need for your analysis. For example, if you only need the order details, sales, and profit, your `SELECT` statement should be `SELECT order_id, order_date, sub_category, sales, profit`. This dramatically reduces the amount of data that BigQuery needs to scan from disk and process.

2.  **Narrow the Partition Filter:** While the original table is not partitioned in this lab, in a real-world scenario, a large table like this would be partitioned by `order_date`. The `WHERE` clause on `order_date` is a good start, as it allows BigQuery to prune partitions (i.e., ignore scanning data from folders that don't match the filter). To optimize further, you should make this filter as specific as possible. Instead of a wide multi-year range, filter to the exact months or days you need, which reduces the number of partitions scanned.

3.  **Use Pre-Aggregated Summary Tables:** The example query scans the raw, granular transaction table. If your analysis frequently involves looking at daily, monthly, or yearly totals by sub-category, it is far more efficient to create a summary table. You could create a table called `daily_sales_by_subcategory` that is updated once a day. Querying this much smaller, pre-aggregated table would be thousands of times faster and cheaper than running a full scan over the raw data every time you need a report.

## Part G — Validation & Counter‑examples (DIVE: Validate)
**Aim:** Avoid “first‑answer fallacy” by testing alternatives.

### G1. Ask for counter‑queries
**Prompt:**
```
I concluded that 'Tables' is a high‑sales but negative‑profit sub-category due to high discounts.
Create two alternative BigQuery SQL queries that could falsify or nuance this finding:
- One that slices by region and time
- One that controls for order priority or ship mode
Return BigQuery SQL only, then a one-paragraph note on how to compare outcomes.
```

In [3]:
print("--- Query 1: Slicing 'Tables' by Region and Year ---")
query_string_1 = """
SELECT
  region,
  EXTRACT(YEAR FROM order_date) AS sales_year,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  AVG(discount) AS avg_discount
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  region,
  sales_year
ORDER BY
  region,
  sales_year;
"""

try:
    # Initialize the BigQuery client to fix the error
    from google.cloud import bigquery
    project_id = 'our-rock-471819-h7'
    client = bigquery.Client(project=project_id)

    query_job_1 = client.query(query_string_1)
    results_df_1 = query_job_1.to_dataframe()
    display(results_df_1)
except Exception as e:
    print(f"An error occurred: {e}")

--- Query 1: Slicing 'Tables' by Region and Year ---


,region,sales_year,total_sales,total_profit,avg_discount
0,Central,2021,7785.4780,-1424.3310,0.326667
1,Central,2022,6857.2600,-265.0939,0.207143
2,Central,2023,13922.9260,292.6211,0.205882
3,Central,2024,10589.3070,-2162.8466,0.292308
4,East,2021,11324.5340,-3629.3475,0.372727
5,East,2022,8904.4060,-2266.2601,0.350000
6,East,2023,7827.5380,-2306.1783,0.352174
7,East,2024,11869.6690,-2884.3612,0.358333
8,South,2021,9940.9445,1107.9902,0.113636
9,South,2022,7370.6745,-2171.3765,0.218750


In [4]:
print("\n--- Query 2: Slicing 'Tables' by Ship Mode ---")
query_string_2 = """
SELECT
  ship_mode,
  SUM(sales) AS total_sales,
  SUM(profit) AS total_profit,
  AVG(discount) AS avg_discount
FROM
  `our-rock-471819-h7.lab1_foundation.superstore_clean`
WHERE
  sub_category = 'Tables'
GROUP BY
  ship_mode
ORDER BY
  total_profit ASC;
"""

try:
    # Initialize the BigQuery client to fix the error
    from google.cloud import bigquery
    project_id = 'our-rock-471819-h7'
    client = bigquery.Client(project=project_id)

    query_job_2 = client.query(query_string_2)
    results_df_2 = query_job_2.to_dataframe()
    display(results_df_2)
except Exception as e:
    print(f"An error occurred: {e}")


--- Query 2: Slicing 'Tables' by Ship Mode ---


,ship_mode,total_sales,total_profit,avg_discount
0,Standard Class,125881.3115,-11937.7372,0.264975
1,Second Class,43693.7475,-3320.6799,0.248361
2,First Class,28800.7760,-1365.3665,0.240426
3,Same Day,9644.3470,-1129.4225,0.261905


### How to Compare Outcomes

To compare the outcomes, run both queries and analyze their results. The first query will reveal if the negative profit for 'Tables' is a universal problem or if it's concentrated in specific regions or years; for example, you might find that only the 'Central' region in 2023 was unprofitable. The second query tests if logistics or customer urgency is a factor, showing if the losses are linked to a specific `ship_mode` like 'Same Day' or 'First Class'. If the initial finding holds true across all slices (all regions, years, and ship modes are unprofitable), the conclusion is robust; otherwise, the problem is more nuanced and the solution should be targeted at the specific unprofitable segments you've identified.

## Part H — Synthesis (DIVE: Extend)
**Aim:** Turn analysis into business‑ready insights.

### H1. Executive‑style summary
**Prompt:**
```
Act as a business strategist.
Based on the following metrics/figures (briefly summarize your results here), write a 4-sentence executive summary:
- 1 sentence: what changed and by how much
- 1 sentence: why it likely changed (drivers)
- 1 sentence: recommended action (who/what/when)
- 1 sentence: metric to monitor next
```

The 'Tables' sub-category is consistently unprofitable, generating a total loss exceeding $17,700, despite significant sales volume. This loss is primarily driven by an aggressive discounting strategy, with average discounts exceeding 25-35% in the heavily impacted East and Central regions. We recommend that the Regional Sales Managers for the East and Central regions immediately implement a cap on all discounts for the 'Tables' sub-category, starting this quarter. To track the effectiveness of this change, we will monitor the `total_profit` and `avg_discount` for 'Tables' in these specific regions on a monthly basis.

### H2. Convert final SQL into an automated job (optional)
**Prompt (use only after your SQL is final):**
```
Convert my final BigQuery SQL into a Python script that can run as a scheduled job from Colab or Cloud Functions.
Requirements:
- Use python‑bigquery client
- Parameterize date range
- Write results to a destination table `[YOUR_PROJECT].analytics.outputs_kpi`
- Add basic error handling & logging
Return one complete runnable script.
```

---
## Submission checklist
- [ ] Kept prompts precise and reproducible  
- [ ] Captured at least **one** CTE query and **one** window function query  
- [ ] Documented **two** validation attempts (counter‑queries or alternate slice)  
- [ ] Wrote a 4‑sentence executive summary based on results  
- [ ] (Optional) Converted final query into a scheduled job
---